from https://askubuntu.com/questions/526747/setting-up-raid-1-on-14-04-with-an-existing-drive

It is possible, but know that it will take a bit of time for the mirror to finish since we are talking about 3 TB. What I would do is:

First prepare the disk with `gdisk` since `fdisk` can not do a partition larger than 2TB. Let us assume that the partition created was `sdb1`

Run `sudo mdadm --create --verbose /dev/md0 --force --level=1 --raid-devices=1 /dev/sdb1`. The important part here is to tell `mdadm` that you are using `--raid-devices=1` one disk right now for the RAID Level 1.

Format the newly created RAID partition and copy everything from the original disk to this partition.

Now prepare the other drive to be included to the RAID by issuing the following command (Assuming the original drive is `sda1`:

`sudo mdadm --grow /dev/md0 --raid-devices=2 --add /dev/sda1`

This will grow the RAID Array (md0), set the amount of devices on it to 2 (sdb1, sda1) and add the original sda1 to it.

You can check the process of it with sudo mdadm --detail /dev/md0 it should show something like this at the end:

```
Number   Major   Minor   RaidDevice State
   0       8       49        0      active sync   /dev/sdb1
   1       8       65        1      spare rebuilding   /dev/sda1
```

And after it is finished:

```
Number   Major   Minor   RaidDevice State
   0       8       49        0      active sync   /dev/sdb1
   1       8       65        1      active sync   /dev/sda1
```

I recommend a reboot after this to make Nautilus see the RAID correctly. You can verify this is working but literally removing one of the drives while having the unit mounted and still being able to execute or view a video in it. Of course the output of `--detail` above would be something like:

```
Number   Major   Minor   RaidDevice State
   0       0        0        0      removed
   1       8       65        1      active sync   /dev/sda1
```

In this case I disconnected the drive `sdb` and still could work on the information.

UPDATE: Since util-linux 2.24, fdisk not supports partitioning 2+ TB and has support for GPT schemes. This support has landed in Ubuntu 14.10. Apart from this, the interface has gotten much friendlier and it supports better human readable sizes. The overall changes can be found in:

util-linux 2.24 - Release Notes
util-linux 2.25 - Release Notes